# Building an Image Classifier and Integrating it into Web Applications Using AI Squared

In this demo, we are going to create an image classification model using TensorFlow and BeyondML and integrate it into an `.air` file to be consumed using the AI Squared Platform and Browser Extension

To do this, we are first going to install the `aisquared` and `beyondml` packages from `PyPI`, then we are going to use the `CIFAR10` dataset from TensorFlow to build an image classification model.

In [1]:
! pip install aisquared
! pip install beyondml

In [2]:
import tensorflow as tf
from beyondml import tflow
import aisquared
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train/255
x_test = x_test/255

In [4]:
model = tf.keras.applications.MobileNetV3Small(weights = None, include_top = True, input_shape = x_train.shape[1:], classes = 10)

#model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
#model = tflow.utils.add_layer_masks(model)
#model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
#model = tflow.utils.mask_model(
#    model,
#    percentile = 75,
#    x = x_train[:1000],
#    y = y_train[:1000].reshape(-1, 1)
#)
model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [5]:
model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 16, 16, 16)   432         ['rescaling[0][0]']              
                                                                                                  
 Conv/BatchNorm (BatchNormaliza  (None, 16, 16, 16)  64          ['Conv[0][0]']                   
 tion)                                                                             

                                                                 Norm[0][0]']                     
                                                                                                  
 expanded_conv_1/project (Conv2  (None, 4, 4, 24)    1728        ['re_lu_4[0][0]']                
 D)                                                                                               
                                                                                                  
 expanded_conv_1/project/BatchN  (None, 4, 4, 24)    96          ['expanded_conv_1/project[0][0]']
 orm (BatchNormalization)                                                                         
                                                                                                  
 expanded_conv_2/expand (Conv2D  (None, 4, 4, 88)    2112        ['expanded_conv_1/project/BatchNo
 )                                                               rm[0][0]']                       
          

 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    0           ['expanded_conv_3/squeeze_excite/
 /Relu (ReLU)                                                    Conv[0][0]']                     
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 96)    2400        ['expanded_conv_3/squeeze_excite/
 /Conv_1 (Conv2D)                                                Relu[0][0]']                     
                                                                                                  
 tf.__operators__.add_4 (TFOpLa  (None, 1, 1, 96)    0           ['expanded_conv_3/squeeze_excite/
 mbda)                                                           Conv_1[0][0]']                   
                                                                                                  
 re_lu_9 (ReLU)                 (None, 1, 1, 96)     0           ['tf.__operators__.add_4[0][0]'] 
          

 expanded_conv_4/project (Conv2  (None, 2, 2, 40)    9600        ['expanded_conv_4/squeeze_excite/
 D)                                                              Mul[0][0]']                      
                                                                                                  
 expanded_conv_4/project/BatchN  (None, 2, 2, 40)    160         ['expanded_conv_4/project[0][0]']
 orm (BatchNormalization)                                                                         
                                                                                                  
 expanded_conv_4/Add (Add)      (None, 2, 2, 40)     0           ['expanded_conv_3/project/BatchNo
                                                                 rm[0][0]',                       
                                                                  'expanded_conv_4/project/BatchNo
                                                                 rm[0][0]']                       
          

                                                                                                  
 expanded_conv_6/expand/BatchNo  (None, 2, 2, 120)   480         ['expanded_conv_6/expand[0][0]'] 
 rm (BatchNormalization)                                                                          
                                                                                                  
 tf.__operators__.add_11 (TFOpL  (None, 2, 2, 120)   0           ['expanded_conv_6/expand/BatchNor
 ambda)                                                          m[0][0]']                        
                                                                                                  
 re_lu_16 (ReLU)                (None, 2, 2, 120)    0           ['tf.__operators__.add_11[0][0]']
                                                                                                  
 tf.math.multiply_11 (TFOpLambd  (None, 2, 2, 120)   0           ['re_lu_16[0][0]']               
 a)       

 thwiseConv2D)                                                                                    
                                                                                                  
 expanded_conv_7/depthwise/Batc  (None, 2, 2, 144)   576         ['expanded_conv_7/depthwise[0][0]
 hNorm (BatchNormalization)                                      ']                               
                                                                                                  
 tf.__operators__.add_15 (TFOpL  (None, 2, 2, 144)   0           ['expanded_conv_7/depthwise/Batch
 ambda)                                                          Norm[0][0]']                     
                                                                                                  
 re_lu_20 (ReLU)                (None, 2, 2, 144)    0           ['tf.__operators__.add_15[0][0]']
                                                                                                  
 tf.math.m

                                                                                                  
 tf.math.multiply_18 (TFOpLambd  (None, 1, 1, 288)   0           ['re_lu_23[0][0]']               
 a)                                                                                               
                                                                                                  
 multiply_12 (Multiply)         (None, 1, 1, 288)    0           ['expanded_conv_8/depthwise/Batch
                                                                 Norm[0][0]',                     
                                                                  'tf.math.multiply_18[0][0]']    
                                                                                                  
 expanded_conv_8/squeeze_excite  (None, 1, 1, 288)   0           ['multiply_12[0][0]']            
 /AvgPool (GlobalAveragePooling                                                                   
 2D)      

                                                                                                  
 expanded_conv_9/squeeze_excite  (None, 1, 1, 576)   83520       ['expanded_conv_9/squeeze_excite/
 /Conv_1 (Conv2D)                                                Relu[0][0]']                     
                                                                                                  
 tf.__operators__.add_22 (TFOpL  (None, 1, 1, 576)   0           ['expanded_conv_9/squeeze_excite/
 ambda)                                                          Conv_1[0][0]']                   
                                                                                                  
 re_lu_27 (ReLU)                (None, 1, 1, 576)    0           ['tf.__operators__.add_22[0][0]']
                                                                                                  
 tf.math.multiply_22 (TFOpLambd  (None, 1, 1, 576)   0           ['re_lu_27[0][0]']               
 a)       

 expanded_conv_10/squeeze_excit  (None, 1, 1, 576)   0           ['multiply_16[0][0]',            
 e/Mul (Multiply)                                                 'tf.math.multiply_25[0][0]']    
                                                                                                  
 expanded_conv_10/project (Conv  (None, 1, 1, 96)    55296       ['expanded_conv_10/squeeze_excite
 2D)                                                             /Mul[0][0]']                     
                                                                                                  
 expanded_conv_10/project/Batch  (None, 1, 1, 96)    384         ['expanded_conv_10/project[0][0]'
 Norm (BatchNormalization)                                       ]                                
                                                                                                  
 expanded_conv_10/Add (Add)     (None, 1, 1, 96)     0           ['expanded_conv_9/Add[0][0]',    
          

In [6]:
model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    batch_size = 128,
    epochs = 100,
    callbacks = tf.keras.callbacks.EarlyStopping(min_delta = 0.004, patience=5, restore_best_weights = True))

Epoch 1/100


2023-01-24 17:10:57.258012: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


313/313 [==============================] - 34s 93ms/step - loss: 1.8241 - accuracy: 0.3386 - val_loss: 2.3042 - val_accuracy: 0.0952
Epoch 2/100
313/313 [==============================] - 31s 99ms/step - loss: 1.5350 - accuracy: 0.4457 - val_loss: 2.3070 - val_accuracy: 0.1016
Epoch 3/100
313/313 [==============================] - 30s 97ms/step - loss: 1.4361 - accuracy: 0.4850 - val_loss: 2.3175 - val_accuracy: 0.0952
Epoch 4/100
313/313 [==============================] - 32s 103ms/step - loss: 1.3659 - accuracy: 0.5133 - val_loss: 2.3417 - val_accuracy: 0.1016
Epoch 5/100
313/313 [==============================] - 35s 113ms/step - loss: 1.3169 - accuracy: 0.5289 - val_loss: 2.3302 - val_accuracy: 0.1016
Epoch 6/100
313/313 [==============================] - 36s 114ms/step - loss: 1.2715 - accuracy: 0.5480 - val_loss: 2.3614 - val_accuracy: 0.1016


In [7]:
confusion_matrix(y_test, model.predict(x_test).argmax(axis = 1))

313/313 [==============================] - 4s 10ms/step


array([[   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1000,    0,    0,    0,    0,    0,    0,    0]])

In [8]:
import tensorflow as tf